# 🧠 GCFS + Nested CV + Hybrid RF–LightGBM Model for Prostate Cancer Biomarkers

This notebook reproduces the machine learning workflow from the manuscript:

> *Bioinformatics and Machine Learning Integration Reveals a Novel 4-Gene Biomarker Model for Prostate Cancer.*

**Pipeline overview:**
- GCFS (Graph-Convolutional Feature Selection)
- Nested cross-validation (10×5)
- Hybrid RF + LightGBM model with Logistic Regression meta-learner
- 70/30 hold-out internal test
- External validation (e.g., GSE46602 dataset)
- ROC analysis, bootstrap 95% CI (n=1000), and DeLong test

### Expected input files
- `DEG_expression_matrix.csv` : discovery expression matrix, shape **(samples × genes)**
- `labels.csv` : discovery labels with a column named **`class`** (0/1 or Normal/Tumor)
- `validation_expression_matrix.csv` : external validation matrix, same genes if possible
- `validation_labels.csv` : validation labels with a column named **`class`**


In [ ]:
# === Imports ===
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from itertools import product
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, roc_curve, auc
import scipy.stats as st
from dataclasses import dataclass, field

RANDOM_STATE = 42
plt.rcParams['figure.dpi'] = 120


In [ ]:
# === GCFS Feature Selector ===
class GCFS(TransformerMixin, BaseEstimator):
    """Simplified Graph-Convolutional Feature Selection (GCFS)
    Each gene score = variance(gene) + mean(neighborhood expression)
    """
    def __init__(self, edges, top_k=15):
        self.edges = edges
        self.top_k = top_k
        self.graph = nx.Graph()
        self.graph.add_edges_from(edges)
        self.selected_features_ = None

    def fit(self, X, y=None):
        scores = {}
        for gene in X.columns:
            neighbors = list(self.graph.neighbors(gene)) if gene in self.graph else []
            neighbor_mean = X[neighbors].mean(axis=1).mean() if neighbors else 0.0
            scores[gene] = float(np.var(X[gene].values)) + float(neighbor_mean)
        self.selected_features_ = sorted(scores, key=scores.get, reverse=True)[: self.top_k]
        return self

    def transform(self, X):
        if self.selected_features_ is None:
            raise RuntimeError("GCFS must be fit before transform.")
        keep = [g for g in self.selected_features_ if g in X.columns]
        return X[keep].copy()


In [ ]:
# === Helper functions ===
def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        'AUC': roc_auc_score(y_true, y_prob),
        'Accuracy': accuracy_score(y_true, y_pred),
        'Sensitivity': recall_score(y_true, y_pred, pos_label=1),
        'Specificity': recall_score(y_true, y_pred, pos_label=0)
    }

def ParameterGridCompat(param_grid):
    if not param_grid:
        yield {}
        return
    keys = list(param_grid.keys())
    vals = [v if isinstance(v, list) else [v] for v in param_grid.values()]
    for combo in product(*vals):
        yield dict(zip(keys, combo))

def bootstrap_auc_ci(y_true, y_prob, n_boot=1000, random_state=42, alpha=0.05):
    rng = np.random.default_rng(random_state)
    aucs = []
    n = len(y_true)
    for _ in range(n_boot):
        idx = rng.integers(0, n, n)
        try:
            aucs.append(roc_auc_score(y_true[idx], y_prob[idx]))
        except ValueError:
            pass
    aucs = np.array(aucs)
    return np.mean(aucs), (np.percentile(aucs, 2.5), np.percentile(aucs, 97.5))

def delong_roc_test(y_true, y_pred1, y_pred2):
    def compute_midrank(x):
        J = np.argsort(x)
        Z = x[J]
        N = len(x)
        T = np.zeros(N, dtype=float)
        i = 0
        while i < N:
            j = i
            while j < N and Z[j] == Z[i]:
                j += 1
            T[i:j] = 0.5 * (i + j - 1)
            i = j
        T2 = np.empty(N, dtype=float)
        T2[J] = T + 1
        return T2

    def calc_auc_cov(y_true, y_pred):
        y_true = np.array(y_true)
        y_pred = np.array(y_pred)
        pos = y_pred[y_true == 1]
        neg = y_pred[y_true == 0]
        n1, n2 = len(pos), len(neg)
        all_ranks = compute_midrank(np.concatenate([pos, neg]))
        V10 = all_ranks[:n1] - (n1 + 1) / 2
        V01 = all_ranks[n1:] - (n1 + 1) / 2
        auc_val = (V10.mean() - V01.mean()) / (n1 * n2)
        s10 = np.cov(V10)
        s01 = np.cov(V01)
        s = s10 / n1 + s01 / n2
        return auc_val, s

    auc1, s1 = calc_auc_cov(y_true, y_pred1)
    auc2, s2 = calc_auc_cov(y_true, y_pred2)
    se = np.sqrt(s1 + s2)
    z = (auc1 - auc2) / se
    p_val = 2 * (1 - st.norm.cdf(abs(z)))
    return auc1, auc2, z, p_val

def plot_roc_curves(y_true, preds_dict, title="ROC Curves"):
    plt.figure(figsize=(7, 6))
    for name, y_prob in preds_dict.items():
        fpr, tpr, _ = roc_curve(y_true, y_prob)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f"{name} (AUC = {roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlabel('1 - Specificity (FPR)')
    plt.ylabel('Sensitivity (TPR)')
    plt.title(title)
    plt.legend(loc='lower right')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
# === Nested Hybrid Trainer (RF + LGBM + Meta LR) ===
@dataclass
class NestedHybridTrainer:
    ppi_edges: list
    top_k: int = 15
    outer_splits: int = 10
    inner_splits: int = 5
    random_state: int = RANDOM_STATE
    best_params_: dict = field(default_factory=dict)

    def _inner_search(self, base_estimator, X, y, param_grid):
        if not param_grid:
            return {}
        skf = StratifiedKFold(n_splits=self.inner_splits, shuffle=True, random_state=self.random_state)
        best_auc, best_params = -np.inf, {}
        for combo in ParameterGridCompat(param_grid):
            aucs = []
            for tr, va in skf.split(X, y):
                X_tr, X_va = X.iloc[tr], X.iloc[va]
                y_tr, y_va = y.iloc[tr], y.iloc[va]
                pipe = Pipeline([
                    ("gcfs", GCFS(self.ppi_edges, self.top_k)),
                    ("scaler", StandardScaler()),
                    ("clf", clone(base_estimator).set_params(**combo))
                ])
                pipe.fit(X_tr, y_tr)
                prob = pipe.predict_proba(X_va)[:, 1]
                aucs.append(roc_auc_score(y_va, prob))
            mean_auc = np.mean(aucs)
            if mean_auc > best_auc:
                best_auc, best_params = mean_auc, combo
        return best_params

    def fit(self, X_train, y_train):
        # Base models (limited grids as per manuscript spirit)
        models = {
            "rf": RandomForestClassifier(random_state=self.random_state),
            "lgb": LGBMClassifier(random_state=self.random_state)
        }
        grids = {
            "rf":  {"n_estimators": [300, 500], "max_features": ["sqrt", None]},
            "lgb": {"num_leaves": [31, 63], "n_estimators": [100, 200]}
        }

        # Inner search (hyperparameter selection)
        for name, est in models.items():
            self.best_params_[name] = self._inner_search(est, X_train, y_train, grids[name])

        # Outer CV to generate OOF predictions for stacking
        outer = StratifiedKFold(n_splits=self.outer_splits, shuffle=True, random_state=self.random_state)
        oof_rf = np.zeros(len(y_train))
        oof_lgb = np.zeros(len(y_train))

        for tr_idx, va_idx in outer.split(X_train, y_train):
            X_tr, X_va = X_train.iloc[tr_idx], X_train.iloc[va_idx]
            y_tr, y_va = y_train.iloc[tr_idx], y_train.iloc[va_idx]

            rf_pipe = Pipeline([
                ("gcfs", GCFS(self.ppi_edges, self.top_k)),
                ("scaler", StandardScaler()),
                ("clf", RandomForestClassifier(random_state=self.random_state, **self.best_params_["rf"]))
            ])
            lgb_pipe = Pipeline([
                ("gcfs", GCFS(self.ppi_edges, self.top_k)),
                ("scaler", StandardScaler()),
                ("clf", LGBMClassifier(random_state=self.random_state, **self.best_params_["lgb"]))
            ])
            rf_pipe.fit(X_tr, y_tr)
            lgb_pipe.fit(X_tr, y_tr)
            oof_rf[va_idx] = rf_pipe.predict_proba(X_va)[:, 1]
            oof_lgb[va_idx] = lgb_pipe.predict_proba(X_va)[:, 1]

        # Meta-learner on OOF
        meta_X = np.vstack([oof_rf, oof_lgb]).T
        self.meta_ = LogisticRegression(random_state=self.random_state)
        self.meta_.fit(meta_X, y_train)

        # Final base models on full train
        self.rf_final_ = rf_pipe.fit(X_train, y_train)
        self.lgb_final_ = lgb_pipe.fit(X_train, y_train)

    def evaluate(self, X, y, title="Evaluation"):
        rf_pred = self.rf_final_.predict_proba(X)[:, 1]
        lgb_pred = self.lgb_final_.predict_proba(X)[:, 1]
        hybrid_pred = self.meta_.predict_proba(np.vstack([rf_pred, lgb_pred]).T)[:, 1]

        metrics = compute_metrics(y, hybrid_pred)
        mean_auc, (ci_low, ci_high) = bootstrap_auc_ci(y.values, hybrid_pred)
        auc1, auc2, z, p = delong_roc_test(y.values, hybrid_pred, lgb_pred)

        print(f"\n=== {title} ===")
        for k, v in metrics.items():
            print(f"{k}: {v:.4f}")
        print(f"Bootstrap AUC 95% CI: [{ci_low:.3f}, {ci_high:.3f}]")
        print(f"DeLong vs LGBM: z={z:.3f}, p={p:.4f}")

        plot_roc_curves(y, {"Hybrid": hybrid_pred, "RF": rf_pred, "LGBM": lgb_pred}, title=title)
        return metrics


In [ ]:
# === Load data (edit paths if needed) ===
X = pd.read_csv("DEG_expression_matrix.csv", index_col=0)
y = pd.read_csv("labels.csv")["class"]
if y.dtype == object:
    y = y.astype("category").cat.codes

# External validation
X_val = pd.read_csv("validation_expression_matrix.csv", index_col=0)
y_val = pd.read_csv("validation_labels.csv")["class"]
if y_val.dtype == object:
    y_val = y_val.astype("category").cat.codes

# Align columns (genes) between train/test/val if necessary
common_cols = X.columns.intersection(X_val.columns)
X = X[common_cols]
X_val = X_val[common_cols]

# 70/30 split for internal test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=RANDOM_STATE
)

# Example STRING-based PPI edges (extend with your network)
ppi_edges = [
    ("GFUS", "ARHGAP8"),
    ("ARHGAP8", "NBL1"),
    ("NBL1", "ACTB"),
    ("GFUS", "ACTB")
]

trainer = NestedHybridTrainer(ppi_edges=ppi_edges, top_k=15)
trainer.fit(X_train, y_train)

# Internal test evaluation
trainer.evaluate(X_test, y_test, title="Internal Test (Hold-out 30%)")

# External validation evaluation
trainer.evaluate(X_val, y_val, title="External Validation (Independent Dataset)")
